In [ ]:
import os
import numpy as np
import matplotlib.image as mp
#from skimage import img_as_ubyte
from PIL import Image


path = 'E:\\Download\\train_image\\our485\\high\\'    
savepath = 'E:\Download\high\\'   
filelist = os.listdir(path)  # 打开对应的文件夹
num = len(filelist)  #得到文件夹中图像的个数
#print(filelist)
print(num)

for index,imagename in enumerate(filelist):
    NewName = imagename
    NewName = NewName.split('.')
    if index < 10:        
        NewName[0] = str(index+1)
    elif  index>=10 and index <100:
        NewName[0] = str(index+1)
    else:
        NewName[0] = str(index+1)
    if NewName[-1] != 'PNG':
        NewName[-1] = 'PNG'
        NewName = str.join('.',NewName)
        imagename = path + imagename
        NewName = path + NewName
        os.rename(imagename,NewName)
        print(NewName,'修改成功！')
    else:
        NewName[-1] = 'PNG'
        NewName = str.join('.',NewName)
        imagename = path + imagename
        NewName = path + NewName
        os.rename(imagename,NewName)
        print(NewName,'修改成功！') 

In [ ]:
a = os.listdir('E:\\Download\\train_image\\our485\\high\\')
print(len(a))

In [ ]:
import numpy as np
import cv2


def replaceZeroes(data):
    min_nonzero = min(data[np.nonzero(data)])
    data[data == 0] = min_nonzero
    return data
# Retinex算法
def SSR(src_img, size):
    L_blur = cv2.GaussianBlur(src_img, (size, size), 0)
    img = replaceZeroes(src_img)
    L_blur = replaceZeroes(L_blur)

    dst_Img = cv2.log(img/255.0)
    #print(dst_Img)
    dst_Lblur = cv2.log(L_blur/255.0)
    dst_IxL = cv2.multiply(dst_Img,dst_Lblur)
    #print(dst_IxL)
    log_R = cv2.subtract(dst_Img, dst_IxL)
    log_R = np.exp(log_R)
    #print(log_R)
    #dst_R = cv2.normalize(log_R,None,0,255,cv2.NORM_MINMAX)
    #log_uint8 = cv2.convertScaleAbs(dst_R)
    #return log_uint8//1
    return log_R

# 低照度图像合成

In [ ]:
path = 'E:\\Download\\train_image\\our485\\high\\'
save_path = 'E:\\Download\\train_image\\our485\\low2\\'
image_name  = os.listdir(path)
low_rate = np.random.uniform(0,1,len(image_name))
for index,(imgname) in enumerate(image_name):
    img = path + imgname
    print(img)
    size = 3
    src_img = cv2.imread(img)
    b_gray, g_gray, r_gray = cv2.split(src_img)
    b_gray = SSR(b_gray, size)*low_rate[index]
    g_gray = SSR(g_gray, size)*low_rate[index]
    r_gray = SSR(r_gray, size)*low_rate[index]
    img_R = cv2.merge([b_gray, g_gray, r_gray])
    #cv2.imshow('img_R',img_R)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    
    cv2.imwrite(save_path+imgname,img_R*255,[100])
    print(save_path+imgname+'写入成功')

In [ ]:
#low_rate = np.random.uniform(0,1,100)
#print(low_rate)

# 图像切割

In [ ]:
path_1 = 'E:\\Download\\train_image\\our485\\high\\'
path_2 = 'E:\\Download\\train_image\\our485\\low2\\'
savepath = 'E:\\Download\\Retinex_crop\\'
def Crop_image(path_1,path_2,savepath):
    image_1 = os.listdir(path_1)
    image_2 = os.listdir(path_2)
    k = 0
    for image_name in image_1:
        
        image_p1 = path_1 + image_name
        image_p2 = path_2 + image_name
        img1 = cv2.imread(image_p1)
        img2 = cv2.imread(image_p2)
        #print(img1.shape,img2.shape)
        for i in range(10):
            for j in range(15):
                k += 1
                img_1 = img1[i*40:(i+1)*40,j*40:(j+1)*40,:]
                img_2 = img2[i*40:(i+1)*40,j*40:(j+1)*40,:]
                #print(img_1.shape)
                #print(savepath+'high\\'+str(i)+str(j)+'.jpg')
                cv2.imwrite(savepath+'high\\'+str(k)+'.png',img_1)
                cv2.imwrite(savepath+'low\\'+str(k)+'.png',img_2)
        print('图像{}切割保存成功'.format(image_name))
Crop_image(path_1,path_2,savepath)

# 图像路径保存


In [ ]:
from torch.utils.data import Dataset,DataLoader
import pandas as pd
def path_list_txt(root):
    path_list = []
    clas = os.listdir(root)
    name_1 = os.listdir(root +'\\'+ clas[0])
    name_2 = os.listdir(root + '\\' + clas[1])  
    path = list(zip(name_1,name_2))
    np.random.shuffle(path)
    sub_root_1 = root + '\\' + clas[0]
    sub_root_2 = root + '\\' + clas[1]
    f =  open('data.txt','a')
    for index,(img_1,img_2) in enumerate(path):
        #print(img_1,img_2)
        #break
        #path_list.append([sub_root_1+'\\'+img_1,sub_root_2 + '\\'+img_2])
        f.writelines([sub_root_1+'\\'+img_1+'\t'+sub_root_2 + '\\'+img_2+'\n'])
    #print(path)
    f.close()
    print('写入成功')
root = 'E:\Download\Retinex_crop'
path_list_txt(root)

# 数据加载自定义

In [ ]:
from torchvision import transforms,datasets
from   PIL import Image
import torch
import torch.nn as nn
import os
import numpy as np
import cv2
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
import torch.nn.functional as F
class MyDatasets(Dataset):
    def __init__(self,text,transform=None):
        self.imgs = []
        self.labels = []
        file = open(text,'r')
        for i in file:
            img,label = i.split('\t')
            self.imgs.append(img)
            self.labels.append(label)
        self.transform = transform
        
    def __getitem__(self,index):
        image = cv2.imread(self.imgs[index])
        Label = cv2.imread(self.imgs[index])
        image = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
        Label = cv2.cvtColor(Label,cv2.COLOR_BGR2HSV)
        H_H,H_S,H_I = cv2.split(image)
        L_H,L_S,L_I = cv2.split(Label)
        img = H_I
        label = L_I
        if self.transform is not None:
            img = self.transform(img)
            label = self.transform(label)
        else:
            img = torch.FloatTensor(img.reshape(1,40,40))
            label = torch.FloatTensor(label.reshape(1,40,40))
        return img,label
    
    def __len__(self):
        return len(self.imgs)

In [ ]:
text = 'data_Retinex.txt'
batch_size = 64
MyDataset = MyDatasets(text)
train_data = DataLoader(MyDataset,batch_size=batch_size,shuffle = True,drop_last = True)

# 残差网络搭建

In [ ]:
from torchvision import transforms,datasets
from   PIL import Image
import torch
import torch.nn as nn
import os
import numpy as np
import cv2
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
# 残差块定义
class ResBlock(nn.Module):
    def __init__(self,in_channel,out_channel,kernel_size,stride,padding):
        super(ResBlock,self).__init__()
        # 卷积层
        self.cov1 = nn.Conv2d(in_channel,out_channel,kernel_size ,stride,padding)
        # 标准化层
        self.bn1 = nn.BatchNorm2d(out_channel)
        # 卷积层
        self.cov2 = nn.Conv2d(out_channel,out_channel,kernel_size ,stride,padding)
        # 标准话层
        self.bn2 = nn.BatchNorm2d(out_channel)
        # 恒等映射保证和输出维度一致
        if in_channel != out_channel:
            self.identity = nn.Sequential(
             nn.Conv2d(in_channel,out_channel,kernel_size ,stride,padding),
             nn.BatchNorm2d(out_channel)
            )
        else:
            self.identity = nn.Sequential()
    # 前向传播    
    def forward(self,x):
        out = F.relu(self.cov1(x))    # relu激活函数
        out = self.bn1(out)           #标准化
        out = F.relu(self.cov2(out))  # relu激活函数
        out = self.bn2(out)           # 标准话
        out = self.identity(x) + out  # 短接层恒等映射实现
        out = F.relu(out)             # relu激活
        return out
        

In [ ]:
#残差网络实现
class ResNet(nn.Module): #继承nn.Module类  
    def __init__(self):
        super(ResNet,self).__init__()  #调用父类主方法
        self.module = nn.Sequential(
            nn.Conv2d(1,64,3,stride = 1,padding = 1), #卷积层
            nn.ReLU(),                                #ReLU激活
            nn.BatchNorm2d(64),                       #标准化
            ResBlock(64,64,3,1,1),                    #残差块
            ResBlock(64,64,3,1,1),                    #残差块
            ResBlock(64,64,3,1,1),
            nn.Conv2d(64,1,3,1,1),                    #卷积层
            nn.ReLU()
        )
    
    def forward(self,x):                             #前向传播
        return self.module(x)

In [ ]:
epochs = 35
device = torch.device('cuda:0')  #设置启用设备GPU编号0 
CovResNet = ResNet().to(device)  #将残差网络从CPU搬运到GPU
CovResNet.load_state_dict(torch.load('DRCNN_45epoch.pr'))
loss_fun = nn.MSELoss().to(device) #均方误差损失函数
optimizer = optim.Adam(CovResNet.parameters()) #使用Adam优化器参数为网络所有参数

In [ ]:
print(len(train_data))

In [ ]:
loss_list = []
def Running():
    for epoch in range(epochs):
        lossvalue = 0
        for index,(datas,labels) in enumerate(train_data): #循环提取出训练batch
            #print(len(datas))
            #break
            data = datas.cuda()              #将数据搬运到GPU
            #print(data)
            label = labels.cuda()            #将标签搬运至GPU
            logits = CovResNet(data)         #将数据传递给网络经行前向传播
            loss = loss_fun(logits,label)    #损失函数实现
            lossvalue+=loss.item()           #计算没个epoch的损失和
            optimizer.zero_grad()            #梯度归零
            loss.backward()                  #反向传播计算梯度
            optimizer.step()                 #梯度下降
            print('epoch:{}\tbatch:{}\tave_loss:{}'.format(epoch+1,index+1,loss.item()/len(datas)))
        loss_list.append(lossvalue/len(train_data))
        print('epoch:{}\tave_loss:{}'.format(epoch+1,lossvalue/(len(train_data)*64)))

In [19]:
Running()

epoch:7	batch:889	ave_loss:0.24709109961986542
epoch:7	batch:890	ave_loss:0.05067570507526398
epoch:7	batch:891	ave_loss:0.12553907930850983
epoch:7	batch:892	ave_loss:0.016076285392045975
epoch:7	batch:893	ave_loss:0.09960729628801346
epoch:7	batch:894	ave_loss:0.06307730078697205
epoch:7	batch:895	ave_loss:0.043899472802877426
epoch:7	batch:896	ave_loss:0.05493602156639099
epoch:7	batch:897	ave_loss:0.03819674998521805
epoch:7	batch:898	ave_loss:0.025364426895976067
epoch:7	batch:899	ave_loss:0.018990594893693924
epoch:7	batch:900	ave_loss:0.03593064099550247
epoch:7	batch:901	ave_loss:0.12320663779973984
epoch:7	batch:902	ave_loss:0.050720419734716415
epoch:7	batch:903	ave_loss:0.027436360716819763
epoch:7	batch:904	ave_loss:0.06765148788690567
epoch:7	batch:905	ave_loss:0.38849613070487976
epoch:7	batch:906	ave_loss:0.03782578557729721
epoch:7	batch:907	ave_loss:0.05392042547464371
epoch:7	batch:908	ave_loss:0.1466081291437149
epoch:7	batch:909	ave_loss:0.06428303569555283
epoch:7	

KeyboardInterrupt: 

In [20]:
path = 'DRCNN_50epoch.pr'
torch.save(CovResNet.state_dict(),path)


In [ ]:
print(loss_list)

In [ ]:
import pandas as pd
data = pd.Series(loss_list)
data.to_csv('epochsloss.csv',index = False,header = False)